In [1]:
import pandas as pd
import sqlite3

In [2]:
# Creating database connection
conn = sqlite3.connect("sales.db")

In [3]:
# Checking tables in the database
tables = pd.read_sql_query("select name from sqlite_master where type='table'", conn)
tables

,name
0,pizza_sales


In [ ]:
# Iterates through all database tables to display 
# their record count and a preview of the first few rows 
# for data validation.

for table in tables["name"]:
    print('-' * 50, table, '-' * 50)

    count_df = pd.read_sql(
        f"SELECT COUNT(*) AS count FROM {table}",
        conn
    )
    print("Count of records:", count_df["count"].iloc[0])

    display(
        pd.read_sql(
            f"SELECT * FROM {table} LIMIT 5",
            conn
        )
    )


-------------------------------------------------- pizza_sales --------------------------------------------------
Count of records: 48620


,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01-01-2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01-01-2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01-01-2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,01-01-2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,01-01-2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


## KPI´S

In [ ]:
# Calculates the total revenue by summing the total_price 
# column from the pizza_sales table.

Total_Revenue = pd.read_sql_query("""
    SELECT SUM(total_price) AS Total_Revenue FROM pizza_sales
                                  """, conn)
Total_Revenue

,Total_Revenue
0,817860.05


In [ ]:
# Computes the average order value by dividing total revenue 
# by the number of distinct orders.

Average_Order_Value = pd.read_sql_query("""
    SELECT (SUM(total_price) / COUNT(DISTINCT order_id)) 
    AS Avg_Order_Value
    FROM pizza_sales
                                  """, conn)
Average_Order_Value

,Avg_Order_Value
0,38.307262


In [ ]:
# Calculates the total number of pizzas sold by 
# summing the quantity column.

Total_Pizzas_Sold = pd.read_sql_query("""
    SELECT SUM(quantity) AS Total_Pizza_Sold FROM pizza_sales
                                  """, conn)
Total_Pizzas_Sold

,Total_Pizza_Sold
0,49574


In [ ]:
# Counts the total number of unique orders placed 
# in the pizza_sales table.

Total_Orders = pd.read_sql_query("""
    SELECT COUNT(DISTINCT order_id) AS Total_Orders FROM pizza_sales
                                  """, conn)
Total_Orders

,Total_Orders
0,21350


In [ ]:
# Calculates the average number of pizzas per order by 
# dividing total quantity sold by the number 
# of distinct orders.

Average_Pizzas_Per_Order = pd.read_sql_query("""
    SELECT
        CAST(
            CAST(SUM(quantity) AS DECIMAL(18,4)) /
            CAST(COUNT(DISTINCT order_id) AS DECIMAL(18,4))
        AS DECIMAL(18,4)
        ) AS Avg_Pizzas_per_order
    FROM pizza_sales
""", conn)
Average_Pizzas_Per_Order


,Avg_Pizzas_per_order
0,2


## Daily Trend for Total Orders

In [34]:
Daily_Trends = pd.read_sql_query("""
    SELECT
        strftime(
            '%w',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        ) AS weekday_num,

        strftime(
            '%A',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        ) AS order_day,

        COUNT(DISTINCT order_id) AS total_orders,

        CASE strftime(
            '%w',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        )
            WHEN '0' THEN 'Sunday'
            WHEN '1' THEN 'Monday'
            WHEN '2' THEN 'Tuesday'
            WHEN '3' THEN 'Wednesday'
            WHEN '4' THEN 'Thursday'
            WHEN '5' THEN 'Friday'
            WHEN '6' THEN 'Saturday'
        END AS day_name

    FROM pizza_sales
    GROUP BY weekday_num, order_day
    ORDER BY weekday_num
""", conn)

Daily_Trends


,weekday_num,order_day,total_orders,day_name
0,0,None,2624,Sunday
1,1,None,2794,Monday
2,2,None,2973,Tuesday
3,3,None,3024,Wednesday
4,4,None,3239,Thursday
5,5,None,3538,Friday
6,6,None,3158,Saturday


## Monthly Trend for Orders

In [35]:
Monthly_Trends = pd.read_sql_query("""
    SELECT
        strftime(
            '%m',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        ) AS month_num,

        strftime(
            '%B',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        ) AS order_month,

        COUNT(DISTINCT order_id) AS total_orders,

        CASE strftime(
            '%m',
            substr(order_date, 7, 4) || '-' ||
            substr(order_date, 4, 2) || '-' ||
            substr(order_date, 1, 2)
        )
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END AS month_name

    FROM pizza_sales
    GROUP BY month_num, order_month
    ORDER BY month_num
""", conn)

Monthly_Trends

,month_num,order_month,total_orders,month_name
0,01,None,1845,January
1,02,None,1685,February
2,03,None,1840,March
3,04,None,1799,April
4,05,None,1853,May
5,06,None,1773,June
6,07,None,1935,July
7,08,None,1841,August
8,09,None,1661,September
9,10,None,1646,October


## Percentage of sales by category and size

In [41]:
Sales_By_Category = pd.read_sql_query("""
    SELECT 
        pizza_category, 
        CAST(SUM(total_price) AS DECIMAL(10,2)) AS Total_Sales,
        CAST(SUM(total_price) * 100.0 / 
            (SELECT SUM(total_price) FROM pizza_sales) AS DECIMAL(10,2)) AS PCT
    FROM pizza_sales
    GROUP BY pizza_category
    ORDER BY PCT DESC
""", conn)

Sales_By_Category

,pizza_category,Total_Sales,PCT
0,Classic,220053.10,26.905960
1,Supreme,208197.00,25.456311
2,Chicken,195919.50,23.955138
3,Veggie,193690.45,23.682591


In [40]:
Sales_By_Size = pd.read_sql_query("""
    SELECT 
        pizza_size, 
        CAST(SUM(total_price) AS DECIMAL(10,2)) AS Total_Sales, 
        CAST(SUM(total_price) * 100.0 / 
            (SELECT SUM(total_price) FROM pizza_sales) AS DECIMAL(10,2)) AS PCT
    FROM pizza_sales
    GROUP BY pizza_size
    ORDER BY PCT DESC
""", conn)

Sales_By_Size

,pizza_size,Total_Sales,PCT
0,L,375318.70,45.890333
1,M,249382.25,30.492044
2,S,178076.50,21.773468
3,XL,14076.00,1.721077
4,XXL,1006.60,0.123077


In [46]:
Total_Pizzas_Sold_By_Pizza_Category = pd.read_sql_query("""
    SELECT pizza_category, SUM(quantity) as Total_Quantity_Sold
    FROM pizza_sales
    GROUP BY pizza_category
    ORDER BY Total_Quantity_Sold DESC

""", conn)
Total_Pizzas_Sold_By_Pizza_Category

,pizza_category,Total_Quantity_Sold
0,Classic,14888
1,Supreme,11987
2,Veggie,11649
3,Chicken,11050
